# Emotion Detection

### Imports

In [49]:
import re
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gensim

### Configs

In [61]:
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

### 1. Load dos Datasets

##### 1.1. Carregando arquivos em dataframes

In [3]:
df_train = pd.read_csv('ptbr_train.csv')
df_test = pd.read_csv('ptbr_test.csv')

##### 1.2. Pequena análise exploratória

Aqui buscamos apenas ver o formato dos dados e se existe algum faltante.

1.2.1. df_train

In [4]:
df_train.head()

,id,text,anger,disgust,fear,joy,sadness,surprise
0,ptbr_train_track_a_00001,Moça eu fiz uma análise com o Tinder uma vez e TB sou programador. Perdi uma mês analisando toda a dinâmica disso e posso te responder isso só não posso falar o que fiz aqui por que posso ser linc...,0,0,0,0,0,0
1,ptbr_train_track_a_00002,"eles sempre mostram a míseria, em como tudo era um nojo. a comparação deles de um esgoto e o século passado é a mesma.",1,1,0,0,0,0
2,ptbr_train_track_a_00003,"eu nunca quis tanto algo como quero você. jamais passou pela minha cabeça necessitar qualquer coisa, como necessito seus beijos. percebe o que fez comigo? me tornei tudo que eu imaginava que nunca...",0,0,0,1,1,0
3,ptbr_train_track_a_00004,esperando aqui o stf vim me emparedar do eu expressar o direito de liberdade!,1,0,0,0,0,0
4,ptbr_train_track_a_00005,"e no final o PS vai pagar o pato, ja é o 4° treinador depois do JJ, mas a torcida ainda não percebe isso.",1,0,0,0,0,0


In [5]:
df_train.shape


(2226, 8)

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226 entries, 0 to 2225
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2226 non-null   object
 1   text      2226 non-null   object
 2   anger     2226 non-null   int64 
 3   disgust   2226 non-null   int64 
 4   fear      2226 non-null   int64 
 5   joy       2226 non-null   int64 
 6   sadness   2226 non-null   int64 
 7   surprise  2226 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 139.3+ KB


In [7]:
emotions = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

print("----- valores existentes em cada coluna -----")

for emotion in emotions:
    unique_values = df_train[emotion].unique()
    print(f"{emotion}: {sorted(unique_values)}")

----- valores existentes em cada coluna -----
anger: [0, 1]
disgust: [0, 1]
fear: [0, 1]
joy: [0, 1]
sadness: [0, 1]
surprise: [0, 1]


1.2.2. df_test

In [8]:
df_test.head()

,id,text,anger,disgust,fear,joy,sadness,surprise
0,ptbr_test_track_a_00001,"me recuso a sair de casa nesse frio do caralho, só quero minha cama",1,0,0,0,0,0
1,ptbr_test_track_a_00002,tenho desde camisola nova até vestido de festa uhul,0,0,0,1,0,0
2,ptbr_test_track_a_00003,"minha playlist de descobertas veio com trap, rap e quatro kpops masculinos, eu deixei meu amigo usar por um tempinho e a porra da apple ainda não começou a colocar só girlgroup de novo ...",1,0,0,0,0,0
3,ptbr_test_track_a_00004,"Então, acho que ele gosta de homem e nem sabe ainda. Homem com barba.",0,0,0,0,0,0
4,ptbr_test_track_a_00005,"Parabéns, Prof. Fernando. Parabéns às educadoras e aos educadores do Brasil, incansáveis no ofício de ensinar.",0,0,0,1,0,0


In [9]:
df_test.shape

(2226, 8)

In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226 entries, 0 to 2225
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2226 non-null   object
 1   text      2226 non-null   object
 2   anger     2226 non-null   int64 
 3   disgust   2226 non-null   int64 
 4   fear      2226 non-null   int64 
 5   joy       2226 non-null   int64 
 6   sadness   2226 non-null   int64 
 7   surprise  2226 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 139.3+ KB


In [11]:
emotions = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

print("----- valores existentes em cada coluna -----")

for emotion in emotions:
    unique_values = df_test[emotion].unique()
    print(f"{emotion}: {sorted(unique_values)}")

----- valores existentes em cada coluna -----
anger: [0, 1]
disgust: [0, 1]
fear: [0, 1]
joy: [0, 1]
sadness: [0, 1]
surprise: [0, 1]


fazendo a união dos dois datasets para simplificar o pré processamento.

mais na frente, para treinar o modelo, será possível escolher a divisão entre dataset de treino e teste.

In [12]:
# juntando os dois dataframes
df = pd.concat([df_train, df_test], ignore_index=True)

print(f"Dataset combinado: {df.shape}")
print(f"Original train: {df_train.shape}")
print(f"Original test: {df_test.shape}")
df.head()

Dataset combinado: (4452, 8)
Original train: (2226, 8)
Original test: (2226, 8)


,id,text,anger,disgust,fear,joy,sadness,surprise
0,ptbr_train_track_a_00001,Moça eu fiz uma análise com o Tinder uma vez e TB sou programador. Perdi uma mês analisando toda a dinâmica disso e posso te responder isso só não posso falar o que fiz aqui por que posso ser linc...,0,0,0,0,0,0
1,ptbr_train_track_a_00002,"eles sempre mostram a míseria, em como tudo era um nojo. a comparação deles de um esgoto e o século passado é a mesma.",1,1,0,0,0,0
2,ptbr_train_track_a_00003,"eu nunca quis tanto algo como quero você. jamais passou pela minha cabeça necessitar qualquer coisa, como necessito seus beijos. percebe o que fez comigo? me tornei tudo que eu imaginava que nunca...",0,0,0,1,1,0
3,ptbr_train_track_a_00004,esperando aqui o stf vim me emparedar do eu expressar o direito de liberdade!,1,0,0,0,0,0
4,ptbr_train_track_a_00005,"e no final o PS vai pagar o pato, ja é o 4° treinador depois do JJ, mas a torcida ainda não percebe isso.",1,0,0,0,0,0


### 2. Limpeza dos Textos

Processo de limpeza utilizado no _sentiment.py_

In [13]:
# Punctuation list
punctuations = re.escape('!"#%\'()*+,./:;<=>?@[\\]^_`{|}~')

# ##### #
# Regex #
# ##### #
re_remove_brackets = re.compile(r'\{.*\}')
re_remove_html = re.compile(r'<(\/|\\)?.+?>', re.UNICODE)
re_transform_numbers = re.compile(r'\d', re.UNICODE)
re_transform_emails = re.compile(r'[^\s]+@[^\s]+', re.UNICODE)
re_transform_url = re.compile(r'(http|https)://[^\s]+', re.UNICODE)
re_transform_username = re.compile(r'@[^\s]+', re.UNICODE)
# Different quotes are used.
re_quotes_1 = re.compile(r"(?u)(^|\W)[‘’′`']", re.UNICODE)
re_quotes_2 = re.compile(r"(?u)[‘’`′'](\W|$)", re.UNICODE)
re_quotes_3 = re.compile(r'(?u)[‘’`′“”]', re.UNICODE)
re_dots = re.compile(r'(?<!\.)\.\.(?!\.)', re.UNICODE)
re_punctuation = re.compile(r'([,";:]){2},', re.UNICODE)
re_hiphen = re.compile(r' -(?=[^\W\d_])', re.UNICODE)
re_tree_dots = re.compile(u'…', re.UNICODE)
# Differents punctuation patterns are used.
re_punkts = re.compile(r'(\w+)([%s])([ %s])' %
                       (punctuations, punctuations), re.UNICODE)
re_punkts_b = re.compile(r'([ %s])([%s])(\w+)' %
                         (punctuations, punctuations), re.UNICODE)
re_punkts_c = re.compile(r'(\w+)([%s])$' % (punctuations), re.UNICODE)
re_changehyphen = re.compile(u'–')
re_doublequotes_1 = re.compile(r'(\"\")')
re_doublequotes_2 = re.compile(r'(\'\')')
re_trim = re.compile(r' +', re.UNICODE)

In [14]:
def clean_text(text):
    """Apply all regex above to a given string."""
    text = text.lower()
    text = text.replace('\xa0', ' ')
    text = text.replace('#', ' ')
    text = re_tree_dots.sub('...', text)
    text = re.sub('\.\.\.', '', text)
    text = re_remove_brackets.sub('', text)
    text = re_changehyphen.sub('-', text)
    text = re_remove_html.sub(' ', text)
    text = re_transform_numbers.sub('0', text)
    text = re_transform_url.sub('URL', text)
    text = re_transform_username.sub('USERNAME', text)
    text = re_transform_emails.sub('EMAIL', text)
    text = re_quotes_1.sub(r'\1"', text)
    text = re_quotes_2.sub(r'"\1', text)
    text = re_quotes_3.sub('"', text)
    text = re.sub('"', '', text)
    text = re_dots.sub('.', text)
    text = re_punctuation.sub(r'\1', text)
    text = re_hiphen.sub(' - ', text)
    text = re_punkts.sub(r'\1 \2 \3', text)
    text = re_punkts_b.sub(r'\1 \2 \3', text)
    text = re_punkts_c.sub(r'\1 \2', text)
    text = re_doublequotes_1.sub('\"', text)
    text = re_doublequotes_2.sub('\'', text)
    text = re_trim.sub(' ', text)
    return text.strip()

In [15]:
df['clean_text'] = np.vectorize(clean_text)(df['text'])

df.head()

,id,text,anger,disgust,fear,joy,sadness,surprise,clean_text
0,ptbr_train_track_a_00001,Moça eu fiz uma análise com o Tinder uma vez e TB sou programador. Perdi uma mês analisando toda a dinâmica disso e posso te responder isso só não posso falar o que fiz aqui por que posso ser linc...,0,0,0,0,0,0,moça eu fiz uma análise com o tinder uma vez e tb sou programador . perdi uma mês analisando toda a dinâmica disso e posso te responder isso só não posso falar o que fiz aqui por que posso ser lin...
1,ptbr_train_track_a_00002,"eles sempre mostram a míseria, em como tudo era um nojo. a comparação deles de um esgoto e o século passado é a mesma.",1,1,0,0,0,0,"eles sempre mostram a míseria , em como tudo era um nojo . a comparação deles de um esgoto e o século passado é a mesma ."
2,ptbr_train_track_a_00003,"eu nunca quis tanto algo como quero você. jamais passou pela minha cabeça necessitar qualquer coisa, como necessito seus beijos. percebe o que fez comigo? me tornei tudo que eu imaginava que nunca...",0,0,0,1,1,0,"eu nunca quis tanto algo como quero você . jamais passou pela minha cabeça necessitar qualquer coisa , como necessito seus beijos . percebe o que fez comigo ? me tornei tudo que eu imaginava que n..."
3,ptbr_train_track_a_00004,esperando aqui o stf vim me emparedar do eu expressar o direito de liberdade!,1,0,0,0,0,0,esperando aqui o stf vim me emparedar do eu expressar o direito de liberdade !
4,ptbr_train_track_a_00005,"e no final o PS vai pagar o pato, ja é o 4° treinador depois do JJ, mas a torcida ainda não percebe isso.",1,0,0,0,0,0,"e no final o ps vai pagar o pato , ja é o 0° treinador depois do jj , mas a torcida ainda não percebe isso ."


In [16]:
from emoji import demojize
df.clean_text = df.clean_text.apply(lambda x: demojize(x, language='pt'))

# remover palavras com menos que 3 letras
df.clean_text = df.clean_text.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))

clean_df = df[['clean_text', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']]

clean_df.head()

,clean_text,anger,disgust,fear,joy,sadness,surprise
0,moça análise tinder programador perdi analisando toda dinâmica disso posso responder isso posso falar aqui posso linchado maluco nada geral funciona desvio comportamento nesses apps quiser detalhe...,0,0,0,0,0,0
1,eles sempre mostram míseria como tudo nojo comparação deles esgoto século passado mesma,1,1,0,0,0,0
2,nunca quis tanto algo como quero você jamais passou pela minha cabeça necessitar qualquer coisa como necessito seus beijos percebe comigo tornei tudo imaginava nunca seria você virou avesso perceb...,0,0,0,1,1,0
3,esperando aqui emparedar expressar direito liberdade,1,0,0,0,0,0
4,final pagar pato treinador depois torcida ainda percebe isso,1,0,0,0,0,0


### 3. Tokenização

Como o texto já está bem limpo, utilizaremos apenas um split para tokenizar as frases.

In [19]:
# separa palavras por espaços
def tokenizar(text):
    return text.split()

clean_df['tokenized_text'] = clean_df.loc[:, 'clean_text'].apply(tokenizar)

clean_df.head(10)

C:\Users\marce\AppData\Local\Temp\ipykernel_28100\2359402247.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['tokenized_text'] = clean_df.loc[:, 'clean_text'].apply(tokenizar)


,clean_text,anger,disgust,fear,joy,sadness,surprise,tokenized_text
0,moça análise tinder programador perdi analisando toda dinâmica disso posso responder isso posso falar aqui posso linchado maluco nada geral funciona desvio comportamento nesses apps quiser detalhe...,0,0,0,0,0,0,"[moça, análise, tinder, programador, perdi, analisando, toda, dinâmica, disso, posso, responder, isso, posso, falar, aqui, posso, linchado, maluco, nada, geral, funciona, desvio, comportamento, ne..."
1,eles sempre mostram míseria como tudo nojo comparação deles esgoto século passado mesma,1,1,0,0,0,0,"[eles, sempre, mostram, míseria, como, tudo, nojo, comparação, deles, esgoto, século, passado, mesma]"
2,nunca quis tanto algo como quero você jamais passou pela minha cabeça necessitar qualquer coisa como necessito seus beijos percebe comigo tornei tudo imaginava nunca seria você virou avesso perceb...,0,0,0,1,1,0,"[nunca, quis, tanto, algo, como, quero, você, jamais, passou, pela, minha, cabeça, necessitar, qualquer, coisa, como, necessito, seus, beijos, percebe, comigo, tornei, tudo, imaginava, nunca, seri..."
3,esperando aqui emparedar expressar direito liberdade,1,0,0,0,0,0,"[esperando, aqui, emparedar, expressar, direito, liberdade]"
4,final pagar pato treinador depois torcida ainda percebe isso,1,0,0,0,0,0,"[final, pagar, pato, treinador, depois, torcida, ainda, percebe, isso]"
5,simm muito famoso aqui mais fora tanto acho aparição dele série certo série solo,0,0,0,0,0,0,"[simm, muito, famoso, aqui, mais, fora, tanto, acho, aparição, dele, série, certo, série, solo]"
6,será esses caras tentar fazer moeda euro sula piada isso fracassar seria algo interessante desse certo querer demais certo,0,0,0,0,0,1,"[será, esses, caras, tentar, fazer, moeda, euro, sula, piada, isso, fracassar, seria, algo, interessante, desse, certo, querer, demais, certo]"
7,acho kkkkk voce nenhuma referencia quem,0,0,0,0,0,0,"[acho, kkkkk, voce, nenhuma, referencia, quem]"
8,obrigada parabéns ainda algumas neste país,0,0,0,1,0,0,"[obrigada, parabéns, ainda, algumas, neste, país]"
9,todo mundo pode fazer parte todos lados tiver escolher pelo menos aproveitar dessas experiências incríveis,0,0,0,1,0,0,"[todo, mundo, pode, fazer, parte, todos, lados, tiver, escolher, pelo, menos, aproveitar, dessas, experiências, incríveis]"


In [21]:
text_tokenizado = clean_df['tokenized_text']
print(text_tokenizado)

0       [moça, análise, tinder, programador, perdi, analisando, toda, dinâmica, disso, posso, responder, isso, posso, falar, aqui, posso, linchado, maluco, nada, geral, funciona, desvio, comportamento, ne...
1                                                                                                         [eles, sempre, mostram, míseria, como, tudo, nojo, comparação, deles, esgoto, século, passado, mesma]
2       [nunca, quis, tanto, algo, como, quero, você, jamais, passou, pela, minha, cabeça, necessitar, qualquer, coisa, como, necessito, seus, beijos, percebe, comigo, tornei, tudo, imaginava, nunca, seri...
3                                                                                                                                                   [esperando, aqui, emparedar, expressar, direito, liberdade]
4                                                                                                                                        [final, pagar, pato, treinador,

### 4. Vetorização

In [22]:
model_w2v = gensim.models.Word2Vec(
            text_tokenizado,
            vector_size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.
            workers= 32, # no.of cores
            seed = 34
)

In [23]:
model_w2v.train(text_tokenizado, total_examples=len(text_tokenizado), epochs=20)

(806713, 1003260)

In [25]:
model_w2v.wv.most_similar(positive="amor")

[('tartaruga', 0.9729823470115662),
 ('deusa', 0.9632256627082825),
 ('trem', 0.955900251865387),
 ('doenças', 0.9556809663772583),
 ('paixão', 0.9549165368080139),
 ('ansiosa', 0.9541447758674622),
 ('genuíno', 0.9532192349433899),
 ('áudio', 0.9517468214035034),
 ('foder', 0.9510223865509033),
 ('bebe', 0.9504831433296204)]

In [26]:
model_w2v.wv.most_similar(positive="ódio")

[('burro', 0.9986252784729004),
 ('explica', 0.9985330700874329),
 ('literalmente', 0.9984830617904663),
 ('estudos', 0.998418927192688),
 ('ganha', 0.9984163045883179),
 ('nova', 0.9983913898468018),
 ('conseguiu', 0.9983706474304199),
 ('embora', 0.9983660578727722),
 ('jovem', 0.9983555674552917),
 ('briga', 0.998324990272522)]

In [27]:
model_w2v.wv.get_vector('amor')

array([-0.14470866, -0.45541322, -0.02048644,  0.15746938, -0.11304978,
       -0.01544372, -0.11926022, -0.12384168,  0.26118597, -0.04888964,
       -0.17659499, -0.1294398 , -0.35089794, -0.23864633, -0.31950277,
        0.07510729,  0.4944026 ,  0.14566113, -0.37211418, -0.1797766 ,
       -0.17559552,  0.08437493, -0.3774579 ,  0.18436296, -0.2654649 ,
        0.3459158 , -0.3334574 , -0.3631899 ,  0.16082165, -0.03788173,
       -0.06971052, -0.45320526, -0.25674665,  0.1927619 , -0.2733896 ,
       -0.26947367,  0.14912292,  0.11779317,  0.05304174, -0.09033505,
       -0.40274993, -0.06172564, -0.24882548,  0.29208252, -0.06158382,
        0.18094477,  0.13648388, -0.07983443, -0.49171466, -0.27347994,
        0.11409485,  0.04424499, -0.02181841,  0.21842337,  0.01447927,
        0.1286437 , -0.48323676, -0.26992467,  0.23445882, -0.15535052,
       -0.05550996, -0.04898844,  0.23243925, -0.24349101,  0.17740363,
        0.07820487,  0.3939105 ,  0.06873906, -0.13445628, -0.24

In [29]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v.wv.get_vector(word).reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec


In [28]:
len(text_tokenizado)

4452

In [30]:
wordvec_arrays = np.zeros((len(text_tokenizado), 200))

for i in range(len(text_tokenizado)):
    wordvec_arrays[i,:] = word_vector(text_tokenizado[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(4452, 200)

In [32]:
wordvec_df

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.023187,-0.322147,0.060546,-0.020154,-0.165480,-0.000263,-0.125780,-0.069344,0.187345,0.052779,...,0.122163,-0.156615,0.015454,-0.156243,0.198786,0.071634,-0.108398,-0.173871,0.111979,0.215989
1,0.009557,-0.332318,0.053908,-0.021994,-0.172056,0.017725,-0.164330,-0.111967,0.195790,0.057816,...,0.139484,-0.208055,0.020255,-0.183136,0.191001,0.097967,-0.133459,-0.153749,0.130900,0.239054
2,0.010050,-0.385376,0.076971,-0.027236,-0.205519,-0.007746,-0.168479,-0.097017,0.233463,0.072317,...,0.170930,-0.208081,0.023855,-0.199817,0.245625,0.102432,-0.150376,-0.205888,0.141487,0.258743
3,0.012058,-0.461763,0.082240,-0.025707,-0.232158,0.012537,-0.193342,-0.103785,0.262346,0.080850,...,0.149633,-0.221760,-0.001267,-0.216647,0.264324,0.079770,-0.144667,-0.232373,0.167569,0.298161
4,0.024970,-0.340505,0.076418,-0.021822,-0.173662,-0.010415,-0.134826,-0.074339,0.195851,0.052153,...,0.127908,-0.177805,0.028263,-0.169751,0.210639,0.090505,-0.128609,-0.174665,0.116326,0.225012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4447,0.074116,-0.375509,0.052865,0.012258,-0.168550,-0.019573,-0.063026,-0.036353,0.193911,0.000030,...,0.136630,-0.109434,0.016118,-0.125050,0.182003,0.056752,-0.119984,-0.181200,0.108164,0.217129
4448,-0.055439,-0.170261,0.126982,-0.133131,-0.180267,0.045707,-0.189793,-0.003747,0.175970,0.175407,...,0.077886,-0.150557,-0.030212,-0.168051,0.241913,0.007945,-0.024889,-0.248408,0.043046,0.154179
4449,-0.028586,-0.437827,0.146537,-0.049120,-0.225447,0.067692,-0.164136,-0.035838,0.216585,0.122320,...,0.125241,-0.190898,0.005941,-0.254514,0.369579,0.053653,-0.037322,-0.390035,0.161677,0.308859
4450,-0.005982,-0.422132,0.031283,0.045412,-0.168093,-0.005071,-0.107680,-0.072129,0.230886,-0.005310,...,0.196088,-0.114533,-0.029603,-0.189465,0.237038,0.107633,-0.142830,-0.217735,0.193501,0.277733


In [35]:
labels = clean_df[['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']]

labels.head(10)

,anger,disgust,fear,joy,sadness,surprise
0,0,0,0,0,0,0
1,1,1,0,0,0,0
2,0,0,0,1,1,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0


### 5. Treinamento dos Modelos de Classificação

In [52]:
# imports sklearn

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

In [53]:
# dividindo aleatoriamente em treinamento e teste

X_train, X_test, y_train, y_test = train_test_split(wordvec_df, labels, test_size=0.2, random_state=42, shuffle=True)

##### 5.1.1 Treinamento com Regressão Logística

In [55]:
reg_log_model = MultiOutputClassifier(LogisticRegression(max_iter=1000, random_state=42))
reg_log_model.fit(X_train, y_train)

,estimator,LogisticRegre...ndom_state=42)
,n_jobs,None
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'


In [56]:
y_predito_reg_log = reg_log_model.predict(X_test)

##### 5.1.2 Avaliação do Modelo de Regressão Logística

In [74]:
# acurácia por emoção

accuracies_rlog = []
for i, emotion in enumerate(emotions):
    acc = accuracy_score(y_test.iloc[:, i], y_predito_reg_log[:, i])
    accuracies_rlog.append(acc)
    print(f"{emotion}: {acc:.3f}")

anger: 0.677
disgust: 0.971
fear: 0.932
joy: 0.763
sadness: 0.853
surprise: 0.929


In [69]:
print(classification_report(y_test, y_predito_reg_log, target_names=emotions))

              precision    recall  f1-score   support

       anger       0.83      0.02      0.03       292
     disgust       0.00      0.00      0.00        26
        fear       0.00      0.00      0.00        61
         joy       0.74      0.17      0.27       237
     sadness       0.25      0.01      0.02       129
    surprise       0.00      0.00      0.00        63

   micro avg       0.72      0.06      0.11       808
   macro avg       0.30      0.03      0.05       808
weighted avg       0.56      0.06      0.10       808
 samples avg       0.05      0.05      0.05       808



##### 5.2.1. Treinamento com Rede Neural (MLP)

In [57]:
mlp_model = MultiOutputClassifier(MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42))
mlp_model.fit(X_train, y_train)

,estimator,MLPClassifier...ndom_state=42)
,n_jobs,None
,hidden_layer_sizes,"(100, ...)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,500


In [58]:
y_predito_mlp = mlp_model.predict(X_test)

##### 5.2.2 Avaliação do Modelo MLP

In [75]:
# acurácia por emoção

accuracies_mlp = []
for i, emotion in enumerate(emotions):
    acc = accuracy_score(y_test.iloc[:, i], y_predito_mlp[:, i])
    accuracies_mlp.append(acc)
    print(f"{emotion}: {acc:.3f}")

anger: 0.685
disgust: 0.970
fear: 0.932
joy: 0.786
sadness: 0.853
surprise: 0.929


In [76]:
print(classification_report(y_test, y_predito_mlp, target_names=emotions))

              precision    recall  f1-score   support

       anger       0.55      0.20      0.30       292
     disgust       0.00      0.00      0.00        26
        fear       0.00      0.00      0.00        61
         joy       0.71      0.33      0.45       237
     sadness       0.43      0.05      0.08       129
    surprise       0.00      0.00      0.00        63

   micro avg       0.62      0.18      0.28       808
   macro avg       0.28      0.10      0.14       808
weighted avg       0.47      0.18      0.25       808
 samples avg       0.16      0.14      0.15       808



##### 5.3. Explorando resultados

Buscando entender a razão dos resultados, como por exemplo

* disgust, fear e surprise com 0 precisão
* algumas precisões mais baixas
* algumas acurácias mais baixas